<a href="https://colab.research.google.com/github/wassim-datascience/Test-python-datascience/blob/main/Test_python_datascience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction:

SensCritique souhaite enrichir l’expérience de ses utilisateurs en leur proposant, lors de la lecture d’une critique, d’accéder à d’autres critiques similaires.
Un tel système de recommandation basé sur le texte vise à améliorer la navigation, à favoriser la découverte de points de vue variés et à renforcer l’engagement sur la plateforme.

L’objectif de ce projet est donc de développer un prototype d’algorithme capable d’identifier des critiques proches les unes des autres, en se basant uniquement sur leur contenu textuel.
La démarche suivie peut se résumer ainsi :

- **Exploration et préparation des données** : nettoyage des critiques, traitement des valeurs manquantes, concaténation des titres et du contenu.  
- **Représentation vectorielle des textes** : utilisation de plusieurs approches, du TF-IDF classique aux représentations par *word embeddings* et *sentence embeddings*.  
- **Mesure de similarité** : calcul de proximités entre critiques, principalement à l’aide de la similarité cosinus.  
- **Évaluation qualitative** : analyse des résultats obtenus en fonction de la pertinence perçue et en veillant à limiter les “faux positifs” (par exemple des rapprochements entre films différents).

Cette étude ne vise pas à livrer un système complet de production, mais à proposer une solution simple, reproductible et évolutive, qui puisse servir de base à un système de recommandation de critiques.


# Chargement + exploration des données:

Dans cette section on propose d'explorer rapidement les données notamment en vu de voir à quoi elles ressemblent et de détécter l'éventuelle présence de valeurs manquantes.

In [1]:
import pandas as pd
from google.colab import files

import numpy as np

# On charge les deux csv dans Google Colab
uploaded = files.upload()

# On charge les deux csv dans un dataFrame chacun
df_fc = pd.read_csv("fightclub_critiques.csv")
df_is = pd.read_csv("interstellar_critiques.csv")


Saving fightclub_critiques.csv to fightclub_critiques (1).csv
Saving interstellar_critiques.csv to interstellar_critiques (1).csv


In [2]:
# On explore un peu les données pour voir à quoi elles ressemblent et voir eventuellement
# la présence ou non de valeur manquante
print(df_fc.head(10))
print(df_fc.shape , df_is.shape)



         id                                                URL  rating  \
0     20761   https://senscritique.com/film/xxx/critique/20761      10   
1  30753423  https://senscritique.com/film/xxx/critique/307...       8   
2  12057488  https://senscritique.com/film/xxx/critique/120...      10   
3  10939046  https://senscritique.com/film/xxx/critique/109...       5   
4   1966438  https://senscritique.com/film/xxx/critique/196...       9   
5   2650063  https://senscritique.com/film/xxx/critique/265...       5   
6  23469107  https://senscritique.com/film/xxx/critique/234...       8   
7    455266  https://senscritique.com/film/xxx/critique/455266       6   
8   2578522  https://senscritique.com/film/xxx/critique/257...      10   
9   1713333  https://senscritique.com/film/xxx/critique/171...       8   

  review_date_creation review_date_last_update  review_hits  \
0       12/03/10 00:00          12/03/10 00:00        35817   
1       28/01/15 09:33          28/01/15 09:33        19867

In [3]:
# chercher les valeurs manquantes pour Fight Club
missing_values_fc = df_fc.isnull().sum()
missing_values_fc.head(20)

,0
id,0
URL,0
rating,0
review_date_creation,0
review_date_last_update,0
review_hits,0
review_content,1
review_title,351
gen_review_like_count,0
username,0


In [4]:
# chercher les valeurs manquantes pour Interstellar
missing_values_is = df_is.isnull().sum()
missing_values_is.head(20)

,0
id,0
URL,0
rating,0
review_date_creation,0
review_date_last_update,0
review_hits,0
review_content,0
review_title,168
gen_review_like_count,0
username,0


Dans le csv Fight Club on remarque la présence de 1 article vide + 351 titres vides.
Dans le csv Interstellar on remrque la présence de 168 titres vide.
On propose de supprimer l'article vide.
Vu qu'on compte mettre le titre et l'article dans une même variable text, on remplacera les titres vides par une chaine de caractère vide en vue de la concaténation.

# Preprocessing des données:

In [5]:
df_fc = df_fc.dropna(subset=["review_content"]) # suppression de l'article vide

# On ajoute colonne 'film'
df_fc["film"] = "Fight Club"
df_is["film"] = "Interstellar"

df_all = pd.concat([df_fc, df_is], ignore_index=True) # concaténation des deux tables

df_all["review_title"] = df_all["review_title"].fillna("") # remplacement des titres vides par ""
df_all["text"] = df_all["review_title"] + " " + df_all["review_content"] # Ajout du titre à l'article
df_all[["review_title", "review_content", "text"]].head() # On regarde le résultat

,review_title,review_content,text
0,Analyse du film et du livre,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre Fight Club sort à ...
1,La consommation identitaire,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire Tout a été déjà di...
2,Sons of Anarchy.,"<p>Qu'une oeuvre aussi folle, aussi inconforta...","Sons of Anarchy. <p>Qu'une oeuvre aussi folle,..."
3,Je suis l'ego démesuré de Jack,<p>Beaucoup pensent que ce film n'est qu'un él...,Je suis l'ego démesuré de Jack <p>Beaucoup pen...
4,Il est temps d'enfreindre les deux premières r...,"<p>Objet d’un véritable culte, Fight Club a ma...",Il est temps d'enfreindre les deux premières r...


In [6]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk

# on charge les stopwords français de NLTK
nltk.download("stopwords")
stop_words = set(stopwords.words("french"))

def clean_text(text):
    # Supprimer HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    # Mettre en minuscules
    text = text.lower()
    # Supprimer tout sauf lettres et espaces
    text = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ\s]", " ", text)
    # Supprimer les stopwords
    tokens = [word for word in text.split() if word not in stop_words]
    return " ".join(tokens)

df_all["text"] = df_all["text"].apply(clean_text)
df_all["text"].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text
0,analyse film livre fight club sort fin écrans ...
1,consommation identitaire tout a déjà dit film ...
2,sons of anarchy oeuvre aussi folle aussi incon...
3,ego démesuré jack beaucoup pensent film éloge ...
4,temps enfreindre deux premières règles objet v...


# Métrique d'évaluation:

Ici on est devant une problematique d'apprentissage non suppérvisé et donc de ce fait on ne peut pas évaluer objectivement nos différents models à moins d'avoir une base de critiques annotés avec la similarités entres les différents articles. Une des approches peut être d'annoter manuellement un petit échantillon de critiques, ce qui est quand meme un travail assez laborieux, et on pense que ce n'est pas forcement ce qui est attendu de notre part lors de ce test. Une deuxieme approche aurait pu être de tester nos résultat avec le systeme de recommendations en vigueur sur le site de sens critique mais vu que on n'a que des extraits de l'ensemble des critiques pour les deux films, une telle approche sera infrutueuse. C'est pour celà que je propose d'utiliser le csv Interstellar pour voir déjà à quel point nos differents models vont nous proposer des critique pas du tout du même film. Cette metrique ne prends biensur pas en compte une vrai similarité de contexte mais peut être une première approche pour évaluer nos différents models.

In [7]:
# Ici on code la fonction accuracy_at_k qui va nous permettre d'évaluer nos différents models

from sklearn.metrics.pairwise import cosine_similarity

def accuracy_at_k(matrix, labels, k=5, n_samples=200):
    """
    matrix : matrice d'embeddings (ex. TF-IDF, FastText, SBERT , CamemBERT)
    labels : liste/colonne avec le film correspondant
    k      : nombre de voisins à recommander
    n_samples : nombre de critiques à tester (aléatoire)
    """
    from random import sample

    idx_sample = sample(range(matrix.shape[0]), n_samples)
    accs = []

    sims = cosine_similarity(matrix)  # matrice NxN de similarités

    for idx in idx_sample:
        # voisins triés du plus proche au moins proche
        neighbors = np.argsort(sims[idx])[::-1]
        neighbors = neighbors[1:k+1]  # enlever soi-même (idx)

        # vérifier combien ont le même film
        correct = sum(labels[n] == labels[idx] for n in neighbors)
        accs.append(correct / k)

    return np.mean(accs)




# Approche TF-IDF:

TF-IDF : compare les critiques mot par mot, pondérés par la fréquence. C'est l'approche la plus basique mais la plus rapide. C'est un bon point de départ pour pouvoir voir comment se comporte notre corpus et ce que peuvent apporter les models plus développés/modernes.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer() # on charge le model TFIDF

tfidf_matrix = vectorizer.fit_transform(df_all["text"]) # on vectorise notre text


In [9]:
# Application de l'accuracy_at_5 à TFIDF

labels = df_all["film"].values
acc_tfidf = accuracy_at_k(tfidf_matrix, labels, k=5, n_samples=200)
print(f"Accuracy@5 intra-film avec TF-IDF : {acc_tfidf:.2f}")

Accuracy@5 intra-film avec TF-IDF : 0.91


Ici on a obtenu un bon résultat avec notre métrique précision à 5 (accuracy_at_5). Ceci s'explique par le fait que tfidf repose exclusivement sur la distribution des mots du vocabulaire dans les articles et de ce fait est fortement biaisé par la présence de certains mots caractéristiques à chaque film (noms acteurs, film...). Cependant on si on avait une base de donnée annoté selon les similarités, ce model ne devrait pas bien performer car il ne tient pas compte du context et de la sémantique des mots.

# Approche Word Embedding: FastText

Word Embeddings (Word2Vec / FastText) : chaque mot → vecteur, puis on fait une moyenne des mots pour représenter une critique. Vu que, à ce qu'on sache, il n'existe pas de model Word2Vec pré-entrainé sur des corpus en Français. On propose d'utiliser FastText qui est déjà une amélioration de Word2Vec tout en ayant une implémentation préentrainé sur du Français.

In [15]:
# On télécharge un model fastext préentrainé sur du français:

!wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
!gunzip cc.fr.300.vec.gz

--2025-09-18 13:58:28--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.167.112.53, 3.167.112.66, 3.167.112.129, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.167.112.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1287757366 (1.2G) [binary/octet-stream]
Saving to: ‘cc.fr.300.vec.gz’

cc.fr.300.vec.gz    100%[===================>]   1.20G   138MB/s    in 16s     

2025-09-18 13:58:45 (76.1 MB/s) - ‘cc.fr.300.vec.gz’ saved [1287757366/1287757366]



In [16]:
# j'installe gensim car il est absent ici
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requi

In [10]:
import gensim
import numpy as np

# Charger FastText (version réduite dans ce notebook car la version entière consomme beaucoup de RAM)
model = gensim.models.KeyedVectors.load_word2vec_format("cc.fr.300.vec", binary=False, limit=200000)
# model = gensim.models.KeyedVectors.load_word2vec_format("cc.fr.300.vec", binary=False)

def document_vector(doc, model):
    words = [w for w in doc.split() if w in model]
    if len(words) == 0:
        return np.zeros(model.vector_size)
    return np.mean([model[w] for w in words], axis=0)

# Construire les vecteurs pour toutes les critiques
doc_vectors_ft = np.array([document_vector(text, model) for text in df_all["text"]])

# Accuracy@5 avec FastText
labels = df_all["film"].values
acc_ft = accuracy_at_k(doc_vectors_ft, labels, k=5, n_samples=200)
print(f"Accuracy@5 intra-film avec FastText : {acc_ft:.2f}")


Accuracy@5 intra-film avec FastText : 0.78


ici FastText réduit m'a donné une précision de 66% et lorsque j'ai executé le model entier (qui est long et dépasse parfois la limite de ma RAM) j'ai obtenu une précision de 72%. Ce résultat, plus faible que l'approche TFIDF sur cette métrique en particulier, ne devrait pas trop nous surprendre, car certes tfidf est plus basique comme approche, mais elle se base sur la similarité de distribution de certain mot du vocabulaire dans les articles. Donc dans des articles où les noms des films et des acteurs sont mentionnés à plusieurs reprise c'est tout à fait prévisible que l'approche tfidf va nous proposer des articles du meme film à chaque fois. Or l'approche FastText qui est une approche Word embedding va surement enlever les noms des acteurs et meme parfois des films (s'ils ne figurent pas dans le vocabulaire). Néanmoins si on avait une base annoté selon les similarités on devrait voir, comme dans les benchmarks dans la littérature, un résultat à l'avantage de FastText.

# Approche Sentence embedding: SBERT

Ici on passe à ce que l'état de l'art propose de mieux pour representer nos articles. A l'opposer de l'approche Word Embedding où on est obligé de faire une moyenne sur les représentations de chaque mot pour représenter nos articles et donc de ce fait on perd la structure et le sens global. Le sentence embedding (ici SBERT) génére un vecteur pour tout l'article en tenant compte de la syntaxe et du context, et de ce fait, on est sensé avoir une forte similarité entre les articles proposé.

Ici on propose d'utiliser un model multilangues préentrainé de SBERT, contenant biensur le Français : "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

In [11]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer

# Charger SBERT multilingue
sbert_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Encoder toutes les critiques
doc_vectors_sbert = sbert_model.encode(df_all["text"].tolist(), show_progress_bar=True)

# Accuracy@5 avec SBERT
labels = df_all["film"].values
acc_sbert = accuracy_at_k(doc_vectors_sbert, labels, k=5, n_samples=200)
print(f"Accuracy@5 intra-film avec SBERT : {acc_sbert:.2f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Accuracy@5 intra-film avec SBERT : 0.94


On remarque, sans surprise que SBERT surperforme tous les autres models (score 93%), sur notre métrique proposé. En plus les critiques proposés sont nettement plus similaires, au sens demandé par le sujet du test, et ceci est dû de façon inhérente à la construction et l'objectif meme du model.  

Vu que le model SBERT proposé est un model multilangues, on se doute bien qu'il a quand meme quelques limites par rapport à un model spécialement concu pour le Français (ex: CamemBERT, FlauBERT)

# CamemBERT:

C’est la version française de BERT, entraînée sur des milliards de mots.

Plus adaptée que SBERT multilingue, car elle capte mieux les subtilités du français.

Le problème c'est que CamemBERT n’est pas directement un model de sentence embedding. il faut soit :

- Faire la moyenne des tokens

- Soit passer par CamemBERT + Sentence-Transformers (Comme ça le sentence-transformers opérera sur une très riche/fine représentation des tokens français tout en effectuant un vrai Sentence Embedding ). On prévoit que ce model va être le mieux adapté à notre problématique poser par le sujet du test.

Camembert Seul:

In [12]:
!pip install -q transformers torch

from transformers import CamembertTokenizer, CamembertModel
import torch
import numpy as np

# Charger tokenizer + modèle CamemBERT
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
camembert = CamembertModel.from_pretrained("camembert-base")
camembert.eval()  # mode évaluation


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

CamembertModel(
  (embeddings): CamembertEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): CamembertEncoder(
    (layer): ModuleList(
      (0-11): 12 x CamembertLayer(
        (attention): CamembertAttention(
          (self): CamembertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): CamembertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
 

In [13]:
def camembert_encode(text, model, tokenizer, max_length=128):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
    # outputs.last_hidden_state : (batch_size, seq_len, hidden_size)
    embeddings = outputs.last_hidden_state.squeeze(0).numpy()
    return np.mean(embeddings, axis=0)  # moyenne sur les tokens


In [14]:
# Limiter à un sous-échantillon si c’est trop lourd au début
texts = df_all["text"].tolist()

doc_vectors_camembert = np.array([camembert_encode(t, camembert, tokenizer) for t in texts])


In [15]:
acc_camembert = accuracy_at_k(doc_vectors_camembert, labels, k=5, n_samples=200)
print(f"Accuracy@5 intra-film avec CamemBERT (moyenne tokens) : {acc_camembert:.2f}")


Accuracy@5 intra-film avec CamemBERT (moyenne tokens) : 0.87


Camembert + transformers:

lors de l'utilisation de cette approche j'ai rencontré plusieurs problèmes pour avoir un code qui fonctionne. Ceci peut être un probleme de ma part pour ne pas avoir trouver où ce situe l'erreur ou un package qui présente encore des manquements notament pour traiter les très longs articles.

# Conclusion:

Ce travail a permis de mettre en place un prototype fonctionnel de système de recommandation de critiques similaires.
Les différentes méthodes testées montrent des résultats contrastés :

- **TF-IDF** offre une solution rapide et efficace, capable de capturer les mots saillants mais limitée lorsqu’il s’agit de saisir le sens profond des phrases.  
- **Word embeddings** enrichissent la représentation en tenant compte des relations sémantiques entre les mots, mais leur performance dépend fortement de la qualité des vecteurs utilisés.  
- **Sentence embeddings** apportent un gain significatif en permettant une représentation contextuelle et globale de chaque critique, souvent plus pertinente pour ce type de tâche.

Même si l’évaluation reste qualitative, les résultats obtenus confirment la faisabilité d’un tel système.  
Des améliorations futures pourraient inclure :
- la mise en place d’un échantillon annoté pour une évaluation quantitative,  
- l’intégration de modèles de type **transformers appliqués sur (BERT, DistilBERT)** pour des représentations plus fines,  
- l’ajout de métadonnées (auteur, note attribuée, date, etc.) pour enrichir la recommandation.

En résumé, cette approche démontre qu’il est possible d’identifier efficacement des critiques similaires à partir de leur contenu textuel et constitue une base solide pour le développement d’un système de recommandation au sein de SensCritique.
